# Notes on this notebook

This notebook is admittedly a mess but is a culmination of several tests attempting to get the keplergl config dictionary running properly, the map not rendering in my IDE, and other bugs I had to work out specific to H3 hex ids and keplergl

In [1]:
from pipelines.tree_data_etl import load_tree_data
import h3

ModuleNotFoundError: No module named 'pipelines'

In [1]:
from pipelines.tree_data_etl import load_tree_data

In [ ]:
import keplergl
keplergl.__version__

In [ ]:
import pandas as pd
from urllib import request
import folium
from folium.plugins import TimestampedGeoJson
import json
from geojson import Point, Feature, FeatureCollection, dump
import pickle
from datetime import datetime, date
import os
import h3
from shapely.geometry import Point
import geopandas as gpd
from keplergl import KeplerGl

In [ ]:
def get_most_recent_upload_date():
    r = request.urlopen('https://www.nycgovparks.org/tree-work-orders/street_tree_planting.csv')

    last_build_date = r.readlines()[5].decode('utf-8').split(',')[1].strip('\n')

    last_build_date = datetime.strptime(last_build_date.strip('"'), '%Y-%m-%d %H:%M:%S')

    return last_build_date

def download_new_data():

    df = pd.read_csv('https://www.nycgovparks.org/tree-work-orders/street_tree_planting.csv', skiprows=7)
    df['CompletedDate'] = pd.to_datetime(df['CompletedDate'])
    df['PlantingSeason'] = pd.to_datetime(df['PlantingSeason'])

    return df

def load_tree_data(save=True, return_data=True):

    today = datetime.now()
    file_names = os.listdir('../data/')
    today_file = f'../data/street_tree_planting_{today.strftime("%Y_%m_%d")}.pkl'

    if os.path.exists(today_file):
        df = pd.read_pickle(today_file)

    elif not os.path.exists(today_file):
        most_recent_web = get_most_recent_upload_date()
        if most_recent_web < today:
            most_recent_file = f'../data/street_tree_planting_{most_recent_web.strftime("%Y_%m_%d")}.pkl'
            if os.path.exists(most_recent_file):
                df = pd.read_pickle(most_recent_file)
            elif not os.path.exists(most_recent_file):
                df = download_new_data()
                if save:
                    df.to_pickle(f'../data/street_tree_planting_{most_recent_web.strftime("%Y_%m_%d")}.pkl')
        elif most_recent_web == today:
            df = get_most_recent_upload_date()
            if save:
                df.to_pickle(today_file)

    if return_data:
        return df

In [ ]:
df = load_tree_data()
df

In [ ]:
h3.geo_to_h3(
    lat=25.32,
    lng=55.46,
    resolution=7
)

In [ ]:
h3_result = lambda id_str: h3.h3_get_resolution(id_str) if h3.h3_is_valid(id_str) else None
h3_result('8843a13687fffff')

In [ ]:
h3_res = 8
def geo_to_h3(row):
    return h3.geo_to_h3(lat=row.lat,lng=row.lng,resolution = h3_res)

In [ ]:
df['hex_id'] = df.apply(geo_to_h3, axis=1)
df

In [ ]:
df[df['WOStatus'] == 'Completed'].value_counts('hex_id').reset_index().rename({'hex_id': 'Hex ID', 0: 'Number of Trees'}, axis=1)

In [ ]:
# config = {
#     "version": "v1",
#     "config": {
#         "visState": {
#             "filters": [],
#             "layers": [
#                 {
#                     "id": "ue3wn9",
#                     "type": "geojson",
#                     "config": {
#                         "dataId": "graffiti",
#                         "label": "graffiti",
#                         "color": [
#                             240,
#                             109,
#                             109
#                         ],
#                         "columns": {
#                             "geojson": "geometry"
#                         },
#                         "isVisible": True,
#                         "visConfig": {
#                             "opacity": 0.7,
#                             "strokeOpacity": 0.77,
#                             "thickness": 0.5,
#                             "strokeColor": [
#                                 231,
#                                 159,
#                                 213
#                             ],
#                             "colorRange": {
#                                 "name": "Number of Trees",
#                                 "type": "sequential",
#                                 "category": "Uber",
#                                 "colors": [
#                                     "#5A1846",
#                                     "#900C3F",
#                                     "#C70039",
#                                     "#E3611C",
#                                     "#F1920E",
#                                     "#FFC300"
#                                 ]
#                             },
#                             "strokeColorRange": {
#                                 "name": "Global Warming",
#                                 "type": "sequential",
#                                 "category": "Uber",
#                                 "colors": [
#                                     "#5A1846",
#                                     "#900C3F",
#                                     "#C70039",
#                                     "#E3611C",
#                                     "#F1920E",
#                                     "#FFC300"
#                                 ]
#                             },
#                             "radius": 10,
#                             "sizeRange": [
#                                 0,
#                                 10
#                             ],
#                             "radiusRange": [
#                                 0,
#                                 50
#                             ],
#                             "heightRange": [
#                                 0,
#                                 500
#                             ],
#                             "elevationScale": 5,
#                             "stroked": False,
#                             "filled": True,
#                             "enable3d": True,
#                             "wireframe": False
#                         },
#                         "hidden": False,
#                         "textLabel": [
#                             {
#                                 "field": None,
#                                 "color": [
#                                     255,
#                                     255,
#                                     255
#                                 ],
#                                 "size": 18,
#                                 "offset": [
#                                     0,
#                                     0
#                                 ],
#                                 "anchor": "start",
#                                 "alignment": "center"
#                             }
#                         ]
#                     },
#                     "visualChannels": {
#                         "colorField": None,
#                         "colorScale": "quantile",
#                         "sizeField": None,
#                         "sizeScale": "linear",
#                         "strokeColorField": None,
#                         "strokeColorScale": "quantile",
#                         "height": {"enabled": True},
#                         "heightField": {"name": "Number of Trees"},
#                         "heightScale": "linear",
#                         "heightEnabled": True,
#                         "radiusField": None,
#                         "radiusScale": "linear"
#                     }
#                 },
#                 {
#                     "id": "2uz5svi",
#                     "type": "geojson",
#                     "config": {
#                         "dataId": "block",
#                         "label": "block",
#                         "color": [
#                             198,
#                             198,
#                             198
#                         ],
#                         "columns": {
#                             "hex_id": "hex_id", "Number of Trees": "Number of Trees"
#                         },
#                         "isVisible": True,
#                         "visConfig": {
#                             "opacity": 0.14,
#                             "strokeOpacity": 0.3,
#                             "thickness": 0.1,
#                             "strokeColor": [
#                                 255,
#                                 254,
#                                 230
#                             ],
#                             "colorRange": {
#                                 "name": "Number of Trees",
#                                 "type": "sequential",
#                                 "category": "Uber",
#                                 "colors": [
#                                     "#5A1846",
#                                     "#900C3F",
#                                     "#C70039",
#                                     "#E3611C",
#                                     "#F1920E",
#                                     "#FFC300"
#                                 ]
#                             },
#                             "strokeColorRange": {
#                                 "name": "Number of Trees",
#                                 "type": "sequential",
#                                 "category": "Uber",
#                                 "colors": [
#                                     "#5A1846",
#                                     "#900C3F",
#                                     "#C70039",
#                                     "#E3611C",
#                                     "#F1920E",
#                                     "#FFC300"
#                                 ]
#                             },
#                             "radius": 10,
#                             "sizeRange": [
#                                 0,
#                                 10
#                             ],
#                             "radiusRange": [
#                                 0,
#                                 50
#                             ],
#                             "heightField": "Number of Trees",
#                             "heightRange": [
#                                 0,
#                                 500
#                             ],
#                             "elevationScale": 5,
#                             "stroked": True,
#                             "filled": False,
#                             "enable3d": False,
#                             "wireframe": False
#                         },
#                         "hidden": False,
#                         "textLabel": [
#                             {
#                                 "field": None,
#                                 "color": [
#                                     255,
#                                     255,
#                                     255
#                                 ],
#                                 "size": 18,
#                                 "offset": [
#                                     0,
#                                     0
#                                 ],
#                                 "anchor": "start",
#                                 "alignment": "center"
#                             }
#                         ]
#                     },
#                     "visualChannels": {
#                         "colorField": None,
#                         "colorScale": "quantile",
#                         "sizeField": None,
#                         "sizeScale": "linear",
#                         "strokeColorField": None,
#                         "strokeColorScale": "quantile",
#                         "heightField": "Number of Trees",
#                         "heightScale": "linear",
#                         "radiusField": None,
#                         "radiusScale": "linear"
#                     }
#                 }
#             ],
#             "interactionConfig": {
#                 "tooltip": {
#                     "fieldsToShow": {
#                         "graffiti": [
#                             "count"
#                         ],
#                         "block": []
#                     },
#                     "enabled": True
#                 },
#                 "brush": {
#                     "size": 0.5,
#                     "enabled": False
#                 },
#                 "geocoder": {
#                     "enabled": False
#                 },
#                 "coordinate": {
#                     "enabled": False
#                 }
#             },
#             "layerBlending": "normal",
#             "splitMaps": [],
#             "animationConfig": {
#                 "currentTime": None,
#                 "speed": 1
#             }
#         },
#         "mapState": {
#             "bearing": 20,
#             "dragRotate": True,
#             "latitude": 40.699206,
#             "longitude": -73.996297,
#             "pitch": 75,
#             "zoom": 10.3,
#             "isSplit": False
#         },
#         "mapStyle": {
#             "styleType": "dark",
#             "topLayerGroups": {
#                 "label": True,
#                 "road": True
#             },
#             "visibleLayerGroups": {
#                 "label": True,
#                 "road": False,
#                 "border": False,
#                 "building": False,
#                 "water": True,
#                 "land": True,
#                 "3d building": False
#             },
#             "threeDBuildingColor": [
#                 9.665468314072013,
#                 17.18305478057247,
#                 31.1442867897876
#             ],
#             "mapStyles": {}
#         }
#     }
# # }

In [ ]:
tree_counts_hex = df[df['WOStatus'] == 'Completed'].value_counts('hex_id').reset_index().rename({'hex_id': 'Hex ID', 0: 'value'}, axis=1)
tree_counts_hex

In [ ]:
tree_counts_hex['value'] = tree_counts_hex['value'].astype(int)

In [ ]:
tree_counts_hex.dtypes

# This works > not height must be size

In [ ]:
config = {
                "version": "v1",
                "config": {
                    "visState": {
                        "layers": [
                            {
                                "id": "tn4tqvj",
                                "type": "hexagonId",
                                "config": {
                                    "dataId": "value",
                                    "label": "Hex ID",
                                    "color": [
                                        221,
                                        178,
                                        124
                                    ],
                                    "highlightColor": [
                                        252,
                                        242,
                                        26,
                                        255
                                    ],
                                    "columns": {
                                        "hex_id": "Hex ID",
                                        "value": "value"
                                    },
                                    "isVisible": True,
                                    "visConfig": {
                                        "opacity": 0.8,
                                        "worldUnitSize": 1,
                                        "colorRange": {
                                            "name": "Uber Viz Diverging 1.5",
                                            "type": "diverging",
                                            "category": "Uber",
                                            "colors": [
                                                "#00939C",
                                                    "#5DBABF",
                                                    "#BAE1E2",
                                                    "#F8C0AA",
                                                    "#DD7755",
                                                    "#C22E00"
                                            ]
                                        },
                                        "coverage": 0.95,
                                        "enable3d": True,
                                        "sizeRange": [
                                            0,
                                            500
                                        ],
                                        "coverageRange": [
                                            0,
                                            1
                                        ],
                                        "elevationScale": 25,
                                        "enableElevationZoomFactor": True
                                    },
                                    "hidden": False,
                                    "textLabel": [
                                        {
                                            "field": None,
                                            "color": [
                                                255,
                                                255,
                                                255
                                            ],
                                            "size": 18,
                                            "offset": [
                                                0,
                                                0
                                            ],
                                            "anchor": "start",
                                            "alignment": "center"
                                        }
                                    ]
                                },
                                "visualChannels": {
                                    "colorField": {
                                        "name": "value",
                                        "type": "integer"
                                    },
                                    "colorScale": "quantile",
                                    "sizeField": {
                                        "name": "value",
                                        "type": "integer"
                                    },
                                    "sizeScale": "linear",
                                    "coverage": 0.95,
                                    "coverageField": None,
                                    "coverageScale": "linear",
                                }
                            }
                        ],
                        "filters": [],
                        "interactionConfig": {
                            "tooltip": {
                                "fieldsToShow": {
                                    "value": [
                                        {
                                            "name": "Hex ID",
                                            "format": None
                                        },
                                        {
                                            "name": "value",
                                            "format": None
                                        }
                                    ]
                                },
                                "compareMode": False,
                                "compareType": "absolute",
                                "enabled": True
                            },
                            "brush": {
                                "size": 0.5,
                                "enabled": True
                            },
                            "geocoder": {
                                "enabled": True
                            },
                            "coordinate": {
                                "enabled": True
                            }
                        },
                        "layerBlending": "normal",
                        "splitMaps": [],
                        "animationConfig": {
                            "currentTime": None,
                            "speed": 1
                        }
                    },
                    "mapState": {
                        "bearing": 20,
                        "dragRotate": True,
                        "latitude": 40.71251981584756,
                        "longitude": -73.97134402172449,
                        "pitch": 75,
                        "zoom": 10,
                        "isSplit": False,
                    },
                    "mapStyle": {
                        "styleType": "dark",
                        "topLayerGroups": {},
                        "visibleLayerGroups": {
                            "label": True,
                            "road": True,
                            "border": False,
                            "building": True,
                            "water": True,
                            "land": True,
                            "3d building": False
                        },
                        "threeDBuildingColor": [
                            9.665468314072013,
                            17.18305478057247,
                            31.1442867897876
                        ],
                        "mapStyles": {}
                    }
                }
            }

In [ ]:
# Load an empty map
from keplergl import KeplerGl
map_1 = KeplerGl(config=config, height=600)
map_1.add_data(tree_counts_hex, name='hex_vals')
map_1

In [ ]:
# map_1.add_data(tree_counts_hex, name='value')
# map_1

In [ ]:
map_1.save_to_html(data={'value': tree_counts_hex}, file_name='Number of Trees No Config.html')#, config=config)

# ^^^^ This Works  ^^^^

https://github.com/keplergl/kepler.gl/issues/526

https://github.com/Thiagobc23/kepler-maps/blob/master/graffiti-config.py

In [ ]:
tree_counts_lat_long = df[df['WOStatus'] == 'Completed'].value_counts('hex_id').reset_index().rename({'hex_id': 'Hex ID', 0: 'value'}, axis=1)
tree_counts_lat_long

In [ ]:
tree_counts_lat_long = tree_counts_lat_long.merge(df[df['WOStatus'] == 'Completed'].groupby('hex_id').last()[['lat', 'lng']].reset_index().rename({'hex_id': 'Hex ID'}, axis=1), on='Hex ID')
tree_counts_lat_long

In [ ]:
map_2 = KeplerGl()
map_2.add_data(tree_counts_lat_long[['lat', 'lng']], name='lat_longs')
map_2

In [ ]:
config = {
                    "version": "v1",
                    "config": {
                        "visState": {
                            "filters": [],
                            "layers": [
                                {
                                    "id": "tivbcgf",
                                    "type": "grid",
                                    "config": {
                                        "dataId": "lat_longs",
                                        "label": "Point",
                                        "color": [
                                            18,
                                            147,
                                            154
                                        ],
                                        "highlightColor": [
                                            252,
                                            242,
                                            26,
                                            255
                                        ],
                                        "columns": {
                                            "lat": "lat",
                                            "lng": "lng"
                                        },
                                        "isVisible": True,
                                        "visConfig": {
                                            "opacity": 0.8,
                                            "worldUnitSize": 1,
                                            "colorRange": {
                                                "name": "Uber Viz Diverging 1.5",
                                                "type": "diverging",
                                                "category": "Uber",
                                                "colors": [
                                                    "#00939C",
                                                    "#5DBABF",
                                                    "#BAE1E2",
                                                    "#F8C0AA",
                                                    "#DD7755",
                                                    "#C22E00"
                                                ]
                                            },
                                            "coverage": 1,
                                            "sizeRange": [
                                                0,
                                                500
                                            ],
                                            "percentile": [
                                                0,
                                                100
                                            ],
                                            "elevationPercentile": [
                                                0,
                                                100
                                            ],
                                            "elevationScale": 18,
                                            "enableElevationZoomFactor": True,
                                            "colorAggregation": "count",
                                            "sizeAggregation": "count",
                                            "enable3d": True
                                        },
                                        "hidden": False,
                                        "textLabel": [
                                            {
                                                "field": None,
                                                "color": [
                                                    255,
                                                    255,
                                                    255
                                                ],
                                                "size": 18,
                                                "offset": [
                                                    0,
                                                    0
                                                ],
                                                "anchor": "start",
                                                "alignment": "center"
                                            }
                                        ]
                                    },
                                    "visualChannels": {
                                        "colorField": None,
                                        "colorScale": "quantile",
                                        "sizeField": None,
                                        "sizeScale": "linear"
                                    }
                                }
                            ],
                            "interactionConfig": {
                                "tooltip": {
                                    "fieldsToShow": {
                                        "lat_longs": []
                                    },
                                    "compareMode": False,
                                    "compareType": "absolute",
                                    "enabled": True
                                },
                                "brush": {
                                    "size": 0.5,
                                    "enabled": False
                                },
                                "geocoder": {
                                    "enabled": False
                                },
                                "coordinate": {
                                    "enabled": False
                                }
                            },
                            "layerBlending": "normal",
                            "splitMaps": [],
                            "animationConfig": {
                                "currentTime": None,
                                "speed": 1
                            }
                        },
                        "mapState": {
                            "bearing": 4.694006309148264,
                            "dragRotate": True,
                            "latitude": 40.64647169551058,
                            "longitude": -74.15135253877554,
                            "pitch": 53.0111841171247,
                            "zoom": 9.451941083083048,
                            "isSplit": False
                        },
                        "mapStyle": {
                            "styleType": "dark",
                            "topLayerGroups": {},
                            "visibleLayerGroups": {
                                "label": True,
                                "road": True,
                                "border": False,
                                "building": True,
                                "water": True,
                                "land": True,
                                "3d building": False
                            },
                            "threeDBuildingColor": [
                                9.665468314072013,
                                17.18305478057247,
                                31.1442867897876
                            ],
                            "mapStyles": {}
                        }
                    }
                }

In [ ]:
map_3 = KeplerGl(config=config)
map_3.add_data(df[['lat', 'lng']], name='lat_longs')
map_3

In [237]:
# map_1.add_data(tree_counts_hex, name='value')
# map_1

In [238]:
map_1.save_to_html(data={'value': tree_counts_hex}, file_name='Number of Trees No Config.html')#, config=config)

Map saved to Number of Trees No Config.html!


# ^^^^ This Works  ^^^^

https://github.com/keplergl/kepler.gl/issues/526

https://github.com/Thiagobc23/kepler-maps/blob/master/graffiti-config.py

In [33]:
tree_counts_lat_long = df[df['WOStatus'] == 'Completed'].value_counts('hex_id').reset_index().rename({'hex_id': 'Hex ID', 0: 'value'}, axis=1)
tree_counts_lat_long

,Hex ID,value
0,882a100157fffff,125
1,882a10010dfffff,91
2,882a1005b9fffff,63
3,882a100dc5fffff,63
4,882a107469fffff,63
...,...,...
558,882a107527fffff,1
559,882a107501fffff,1
560,882a103951fffff,1
561,882a1039c9fffff,1


In [40]:
tree_counts_lat_long = tree_counts_lat_long.merge(df[df['WOStatus'] == 'Completed'].groupby('hex_id').last()[['lat', 'lng']].reset_index().rename({'hex_id': 'Hex ID'}, axis=1), on='Hex ID')
tree_counts_lat_long

,Hex ID,value,lat,lng
0,882a100157fffff,125,40.868989,-73.840153
1,882a10010dfffff,91,40.884259,-73.854384
2,882a1005b9fffff,63,40.677720,-73.740996
3,882a100dc5fffff,63,40.710596,-73.937014
4,882a107469fffff,63,40.614911,-73.991221
...,...,...,...,...
558,882a107527fffff,1,40.602977,-74.097827
559,882a107501fffff,1,40.609755,-74.063037
560,882a103951fffff,1,40.613430,-73.820614
561,882a1039c9fffff,1,40.584737,-73.813229


In [195]:
map_2 = KeplerGl()
map_2.add_data(tree_counts_lat_long[['lat', 'lng']], name='lat_longs')
map_2

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'lat_longs': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2…

In [197]:
config = {
                    "version": "v1",
                    "config": {
                        "visState": {
                            "filters": [],
                            "layers": [
                                {
                                    "id": "tivbcgf",
                                    "type": "grid",
                                    "config": {
                                        "dataId": "lat_longs",
                                        "label": "Point",
                                        "color": [
                                            18,
                                            147,
                                            154
                                        ],
                                        "highlightColor": [
                                            252,
                                            242,
                                            26,
                                            255
                                        ],
                                        "columns": {
                                            "lat": "lat",
                                            "lng": "lng"
                                        },
                                        "isVisible": True,
                                        "visConfig": {
                                            "opacity": 0.8,
                                            "worldUnitSize": 1,
                                            "colorRange": {
                                                "name": "Uber Viz Diverging 1.5",
                                                "type": "diverging",
                                                "category": "Uber",
                                                "colors": [
                                                    "#00939C",
                                                    "#5DBABF",
                                                    "#BAE1E2",
                                                    "#F8C0AA",
                                                    "#DD7755",
                                                    "#C22E00"
                                                ]
                                            },
                                            "coverage": 1,
                                            "sizeRange": [
                                                0,
                                                500
                                            ],
                                            "percentile": [
                                                0,
                                                100
                                            ],
                                            "elevationPercentile": [
                                                0,
                                                100
                                            ],
                                            "elevationScale": 18,
                                            "enableElevationZoomFactor": True,
                                            "colorAggregation": "count",
                                            "sizeAggregation": "count",
                                            "enable3d": True
                                        },
                                        "hidden": False,
                                        "textLabel": [
                                            {
                                                "field": None,
                                                "color": [
                                                    255,
                                                    255,
                                                    255
                                                ],
                                                "size": 18,
                                                "offset": [
                                                    0,
                                                    0
                                                ],
                                                "anchor": "start",
                                                "alignment": "center"
                                            }
                                        ]
                                    },
                                    "visualChannels": {
                                        "colorField": None,
                                        "colorScale": "quantile",
                                        "sizeField": None,
                                        "sizeScale": "linear"
                                    }
                                }
                            ],
                            "interactionConfig": {
                                "tooltip": {
                                    "fieldsToShow": {
                                        "lat_longs": []
                                    },
                                    "compareMode": False,
                                    "compareType": "absolute",
                                    "enabled": True
                                },
                                "brush": {
                                    "size": 0.5,
                                    "enabled": False
                                },
                                "geocoder": {
                                    "enabled": False
                                },
                                "coordinate": {
                                    "enabled": False
                                }
                            },
                            "layerBlending": "normal",
                            "splitMaps": [],
                            "animationConfig": {
                                "currentTime": None,
                                "speed": 1
                            }
                        },
                        "mapState": {
                            "bearing": 4.694006309148264,
                            "dragRotate": True,
                            "latitude": 40.64647169551058,
                            "longitude": -74.15135253877554,
                            "pitch": 53.0111841171247,
                            "zoom": 9.451941083083048,
                            "isSplit": False
                        },
                        "mapStyle": {
                            "styleType": "dark",
                            "topLayerGroups": {},
                            "visibleLayerGroups": {
                                "label": True,
                                "road": True,
                                "border": False,
                                "building": True,
                                "water": True,
                                "land": True,
                                "3d building": False
                            },
                            "threeDBuildingColor": [
                                9.665468314072013,
                                17.18305478057247,
                                31.1442867897876
                            ],
                            "mapStyles": {}
                        }
                    }
                }

In [201]:
map_3 = KeplerGl(config=config)
map_3.add_data(df[['lat', 'lng']], name='lat_longs')
map_3

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'tivbcgf', 'type': …

In [9]:
h3.geo_to_h3(df.loc[0, 'lat'], df.loc[0, 'lng'], resolution=8)

'882a10042dfffff'

In [17]:
df.columns

Index(['lng', 'lat', 'Borough', 'ZipCode', 'BuildingNumber', 'StreetName',
       'FiscalYear', 'PlantingSpaceID', 'CommunityBoard', 'PlantingSeason',
       'CityCouncil', 'TreeID', 'WOId', 'WOStatus', 'CompletedDate'],
      dtype='object')

In [24]:
res = 8
# h3.geo_to_h3(df.loc[1, 'lat'], df.loc[1, 'lng'], resolution=res)
# df['hex_id_8'] = df[['lat', 'lng']].apply(lambda x: h3.geo_to_h3(x['lat'], x['lng'], resolution=res))
df[['lat', 'lng']]

,lat,lng
0,40.740340,-73.709503
1,40.735428,-73.712984
2,40.828165,-73.876284
3,40.729696,-73.894586
4,40.815443,-73.957912
...,...,...
7981,40.854817,-73.903583
7982,40.667079,-73.989048
7983,40.665143,-73.990085
7984,40.649622,-74.004315


In [20]:
df[['lat', 'lng']].apply(lambda x: h3.geo_to_h3(x.lat, x.lng, resolution=8))

AttributeError: 'Series' object has no attribute 'lat'

In [25]:
from shapely.geometry import Point # Shapely for converting latitude/longtitude to geometry
import geopandas as gpd # To create GeodataFrame

In [27]:
# creating a geometry column 
geometry = [Point(xy) for xy in zip(df['lng'], df['lat'])]

# Coordinate reference system : WGS84
crs = {'init': 'epsg:4326'}

# Creating a Geographic data frame 
gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
gdf

/Users/msmcmanus/anaconda3/envs/NYC-street-tree-planting/lib/python3.10/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,lng,lat,Borough,ZipCode,BuildingNumber,StreetName,FiscalYear,PlantingSpaceID,CommunityBoard,PlantingSeason,CityCouncil,TreeID,WOId,WOStatus,CompletedDate,geometry
0,-73.709503,40.740340,Queens,11004,82-030,260 STREET,0,148027,413,2022-05-31,23,114029,15525248,Completed,2021-12-07,POINT (-73.70950 40.74034)
1,-73.712984,40.735428,Queens,11001,254-04,84 ROAD,0,177638,413,2022-05-31,23,142438,8573730,Completed,2021-12-07,POINT (-73.71298 40.73543)
2,-73.876284,40.828165,Bronx,10472,1155,MANOR AVENUE,0,191641,209,2022-05-31,18,0,14343624,Not Completed,NaT,POINT (-73.87628 40.82816)
3,-73.894586,40.729696,Queens,11378,69-26,53 DRIVE,0,200074,405,2022-05-31,30,163302,6529475,Completed,2022-05-25,POINT (-73.89459 40.72970)
4,-73.957912,40.815443,Manhattan,0,567,DR M L KING JR BOULEVARD,0,216545,109,2022-05-31,0,183762,13809561,Not Completed,NaT,POINT (-73.95791 40.81544)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7981,-73.903583,40.854817,Bronx,10453,110,EAST 181 STREET,0,432549,205,2022-05-31,14,13134156,13213090,Completed,2022-06-10,POINT (-73.90358 40.85482)
7982,-73.989048,40.667079,Brooklyn,11215,256,13 STREET,0,3644371,306,2022-05-31,39,13145236,14539236,Completed,2021-12-03,POINT (-73.98905 40.66708)
7983,-73.990085,40.665143,Brooklyn,11215,568,5 AVENUE,0,3926932,307,2022-05-31,39,13145237,14539255,Completed,2021-12-03,POINT (-73.99008 40.66514)
7984,-74.004315,40.649622,Brooklyn,11232,531,41 STREET,0,227912,307,2022-05-31,38,13148613,16783658,Completed,2022-05-13,POINT (-74.00432 40.64962)


In [46]:
res = 8
gdf['hex_id'] = gdf['geometry'].apply(lambda x: h3.geo_to_h3(x.y, x.x, resolution=res))
gdf

,lng,lat,Borough,ZipCode,BuildingNumber,StreetName,FiscalYear,PlantingSpaceID,CommunityBoard,PlantingSeason,CityCouncil,TreeID,WOId,WOStatus,CompletedDate,geometry,hex_id
0,-73.709503,40.740340,Queens,11004,82-030,260 STREET,0,148027,413,2022-05-31,23,114029,15525248,Completed,2021-12-07,POINT (-73.70950 40.74034),882a10042dfffff
1,-73.712984,40.735428,Queens,11001,254-04,84 ROAD,0,177638,413,2022-05-31,23,142438,8573730,Completed,2021-12-07,POINT (-73.71298 40.73543),882a100425fffff
2,-73.876284,40.828165,Bronx,10472,1155,MANOR AVENUE,0,191641,209,2022-05-31,18,0,14343624,Not Completed,NaT,POINT (-73.87628 40.82816),882a1001adfffff
3,-73.894586,40.729696,Queens,11378,69-26,53 DRIVE,0,200074,405,2022-05-31,30,163302,6529475,Completed,2022-05-25,POINT (-73.89459 40.72970),882a100c05fffff
4,-73.957912,40.815443,Manhattan,0,567,DR M L KING JR BOULEVARD,0,216545,109,2022-05-31,0,183762,13809561,Not Completed,NaT,POINT (-73.95791 40.81544),882a1008c1fffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7981,-73.903583,40.854817,Bronx,10453,110,EAST 181 STREET,0,432549,205,2022-05-31,14,13134156,13213090,Completed,2022-06-10,POINT (-73.90358 40.85482),882a100ac1fffff
7982,-73.989048,40.667079,Brooklyn,11215,256,13 STREET,0,3644371,306,2022-05-31,39,13145236,14539236,Completed,2021-12-03,POINT (-73.98905 40.66708),882a107741fffff
7983,-73.990085,40.665143,Brooklyn,11215,568,5 AVENUE,0,3926932,307,2022-05-31,39,13145237,14539255,Completed,2021-12-03,POINT (-73.99008 40.66514),882a107747fffff
7984,-74.004315,40.649622,Brooklyn,11232,531,41 STREET,0,227912,307,2022-05-31,38,13148613,16783658,Completed,2022-05-13,POINT (-74.00432 40.64962),882a107701fffff


In [47]:

map_1 = KeplerGl()
map_1.add_data(data=gdf[gdf['WOStatus'] == 'Completed'][['hex_id', 'CompletedDate']], name='data_1')
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data_1': {'index': [0, 1, 3, 10, 12, 13, 15, 36, 39, 40, 41, 42, 71, 78, 80, 83, 84, 104, 105,…

In [42]:
df.rename({'lng': 'Longitude', 'lat': 'Latitude'}, axis=1, inplace=True)
df

,Longitude,Latitude,Borough,ZipCode,BuildingNumber,StreetName,FiscalYear,PlantingSpaceID,CommunityBoard,PlantingSeason,CityCouncil,TreeID,WOId,WOStatus,CompletedDate,geometry,hex_id
0,-73.709503,40.740340,Queens,11004,82-030,260 STREET,0,148027,413,2022-05-31,23,114029,15525248,Completed,2021-12-07,POINT (-73.70950 40.74034),88f05bda3dfffff
1,-73.712984,40.735428,Queens,11001,254-04,84 ROAD,0,177638,413,2022-05-31,23,142438,8573730,Completed,2021-12-07,POINT (-73.71298 40.73543),88f05bda07fffff
2,-73.876284,40.828165,Bronx,10472,1155,MANOR AVENUE,0,191641,209,2022-05-31,18,0,14343624,Not Completed,NaT,POINT (-73.87628 40.82816),88f05aa629fffff
3,-73.894586,40.729696,Queens,11378,69-26,53 DRIVE,0,200074,405,2022-05-31,30,163302,6529475,Completed,2022-05-25,POINT (-73.89459 40.72970),88f05aa65dfffff
4,-73.957912,40.815443,Manhattan,0,567,DR M L KING JR BOULEVARD,0,216545,109,2022-05-31,0,183762,13809561,Not Completed,NaT,POINT (-73.95791 40.81544),88f05ab43bfffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7981,-73.903583,40.854817,Bronx,10453,110,EAST 181 STREET,0,432549,205,2022-05-31,14,13134156,13213090,Completed,2022-06-10,POINT (-73.90358 40.85482),88f05ab597fffff
7982,-73.989048,40.667079,Brooklyn,11215,256,13 STREET,0,3644371,306,2022-05-31,39,13145236,14539236,Completed,2021-12-03,POINT (-73.98905 40.66708),88f05ab685fffff
7983,-73.990085,40.665143,Brooklyn,11215,568,5 AVENUE,0,3926932,307,2022-05-31,39,13145237,14539255,Completed,2021-12-03,POINT (-73.99008 40.66514),88f05ab681fffff
7984,-74.004315,40.649622,Brooklyn,11232,531,41 STREET,0,227912,307,2022-05-31,38,13148613,16783658,Completed,2022-05-13,POINT (-74.00432 40.64962),88f05ab6c7fffff


In [101]:
import keplergl
w1 = keplergl.KeplerGl(height=600, data={'data_1': df[~df['CompletedDate'].isnull()][['Latitude', 'Longitude', 'CompletedDate']]})
# w1.add_data(df[['hex_id', 'TreeID']], name='hexes')
w1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data_1':        Latitude  Longitude CompletedDate
0     40.740340 -73.709503    2021-12-07
1  …

In [48]:
df['hex_id'] = gdf['hex_id']
df

,Longitude,Latitude,Borough,ZipCode,BuildingNumber,StreetName,FiscalYear,PlantingSpaceID,CommunityBoard,PlantingSeason,CityCouncil,TreeID,WOId,WOStatus,CompletedDate,geometry,hex_id
0,-73.709503,40.740340,Queens,11004,82-030,260 STREET,0,148027,413,2022-05-31,23,114029,15525248,Completed,2021-12-07,POINT (-73.70950 40.74034),882a10042dfffff
1,-73.712984,40.735428,Queens,11001,254-04,84 ROAD,0,177638,413,2022-05-31,23,142438,8573730,Completed,2021-12-07,POINT (-73.71298 40.73543),882a100425fffff
2,-73.876284,40.828165,Bronx,10472,1155,MANOR AVENUE,0,191641,209,2022-05-31,18,0,14343624,Not Completed,NaT,POINT (-73.87628 40.82816),882a1001adfffff
3,-73.894586,40.729696,Queens,11378,69-26,53 DRIVE,0,200074,405,2022-05-31,30,163302,6529475,Completed,2022-05-25,POINT (-73.89459 40.72970),882a100c05fffff
4,-73.957912,40.815443,Manhattan,0,567,DR M L KING JR BOULEVARD,0,216545,109,2022-05-31,0,183762,13809561,Not Completed,NaT,POINT (-73.95791 40.81544),882a1008c1fffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7981,-73.903583,40.854817,Bronx,10453,110,EAST 181 STREET,0,432549,205,2022-05-31,14,13134156,13213090,Completed,2022-06-10,POINT (-73.90358 40.85482),882a100ac1fffff
7982,-73.989048,40.667079,Brooklyn,11215,256,13 STREET,0,3644371,306,2022-05-31,39,13145236,14539236,Completed,2021-12-03,POINT (-73.98905 40.66708),882a107741fffff
7983,-73.990085,40.665143,Brooklyn,11215,568,5 AVENUE,0,3926932,307,2022-05-31,39,13145237,14539255,Completed,2021-12-03,POINT (-73.99008 40.66514),882a107747fffff
7984,-74.004315,40.649622,Brooklyn,11232,531,41 STREET,0,227912,307,2022-05-31,38,13148613,16783658,Completed,2022-05-13,POINT (-74.00432 40.64962),882a107701fffff


In [53]:
w2 = keplergl.KeplerGl(height=600)
w2.add_data(data=df[~df['CompletedDate'].isnull()].rename({'CompletedDate':'value'}, axis=1)[['hex_id', 'value']],
           name='data_2')
w2

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data_2': {'index': [0, 1, 3, 10, 12, 13, 15, 36, 39, 40, 41, 42, 71, 78, 80, 83, 84, 104, 105,…

In [57]:
df[~df['CompletedDate'].isnull()].rename({'CityCouncil':'value'}, axis=1)[['hex_id', 'value']].to_csv('test_hex.csv', index=False)

In [58]:
with open('test_hex.csv', 'r') as f:
    csvData = f.read()
    
map_3 = keplergl.KeplerGl(height=600)
map_3.add_data(data=csvData, name='csv_1')

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [59]:
map_3

KeplerGl(data={'csv_1': 'hex_id,value\n882a10042dfffff,23\n882a100425fffff,23\n882a100c05fffff,30\n882a100f23f…

In [60]:
csvData

'hex_id,value\n882a10042dfffff,23\n882a100425fffff,23\n882a100c05fffff,30\n882a100f23fffff,22\n882a107769fffff,0\n882a10776dfffff,38\n882a100d4bfffff,26\n882a100517fffff,23\n882a100ca5fffff,37\n882a107717fffff,44\n882a107549fffff,43\n882a1001d7fffff,13\n882a100c37fffff,34\n882a100c43fffff,25\n882a100141fffff,12\n882a1001c1fffff,13\n882a107445fffff,44\n882a10010bfffff,12\n882a100ee9fffff,24\n882a100ee9fffff,24\n882a100ee9fffff,24\n882a100c37fffff,34\n882a100c37fffff,34\n882a1001e5fffff,18\n882a1001e5fffff,18\n882a1001e5fffff,18\n882a107739fffff,44\n882a100d25fffff,3\n882a100f2bfffff,22\n882a100425fffff,23\n882a100147fffff,12\n882a1001d5fffff,13\n882a1001d7fffff,13\n882a1001e3fffff,18\n882a100ad5fffff,16\n882a100a9dfffff,16\n882a100f03fffff,22\n882a100d53fffff,26\n882a100d53fffff,26\n882a100c43fffff,21\n882a103a67fffff,31\n882a103a67fffff,31\n882a1001bdfffff,18\n882a1001d7fffff,13\n882a1001d7fffff,13\n882a106253fffff,49\n882a100f31fffff,22\n882a100f2bfffff,22\n882a100f3dfffff,22\n882a100

In [89]:
import keplergl
w1 = keplergl.KeplerGl(height=600)
w1.add_data(data=df.groupby('hex_id')['CompletedDate'].count().reset_index(), name='data_grouped')
w1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data_grouped': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19…

In [92]:
import keplergl
w1 = keplergl.KeplerGl(height=600)
w1.add_data(data=pd.DataFrame({'hex_id': ['882a1005b5fffff', '882a100d49fffff'], 'value': [1, 2]}), name='small')
w1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'small': {'index': [0, 1], 'columns': ['hex_id', 'value'], 'data': [['882a1005b5fffff', 1], ['8…

In [69]:
w1.add_data(csvData, name='csv_1')

TypeError: KeplerGl.add_data() got an unexpected keyword argument 'value'

In [68]:
help(keplergl.keplergl)

Help on module keplergl.keplergl in keplergl:

NAME
    keplergl.keplergl

CLASSES
    builtins.Exception(builtins.BaseException)
        TraitError
            DataException
    ipywidgets.widgets.domwidget.DOMWidget(ipywidgets.widgets.widget.Widget)
        KeplerGl
    
    class DataException(TraitError)
     |  Method resolution order:
     |      DataException
     |      TraitError
     |      builtins.Exception
     |      builtins.BaseException
     |      builtins.object
     |  
     |  Data descriptors inherited from TraitError:
     |  
     |  __weakref__
     |      list of weak references to the object (if defined)
     |  
     |  ----------------------------------------------------------------------
     |  Methods inherited from builtins.Exception:
     |  
     |  __init__(self, /, *args, **kwargs)
     |      Initialize self.  See help(type(self)) for accurate signature.
     |  
     |  ----------------------------------------------------------------------
     | 

In [73]:
df.columns

Index(['Longitude', 'Latitude', 'Borough', 'ZipCode', 'BuildingNumber',
       'StreetName', 'FiscalYear', 'PlantingSpaceID', 'CommunityBoard',
       'PlantingSeason', 'CityCouncil', 'TreeID', 'WOId', 'WOStatus',
       'CompletedDate', 'geometry', 'hex_id'],
      dtype='object')

,hex_id,CompletedDate
0,882a100003fffff,1
1,882a100007fffff,11
2,882a100039fffff,3
3,882a100101fffff,17
4,882a100103fffff,20
...,...,...
638,882a107757fffff,31
639,882a107759fffff,22
640,882a10775bfffff,37
641,882a107769fffff,1
